In [1]:
library(coin)
library(lme4)
library(lmerTest)
library(plyr)
library(xtable)
library(LMERConvenienceFunctions)
library(car)

Loading required package: survival
Loading required package: Matrix

Attaching package: 'lmerTest'

The following object is masked from 'package:lme4':

    lmer

The following object is masked from 'package:stats':

    step

Loading required package: carData
Registered S3 methods overwritten by 'car':
  method                          from
  influence.merMod                lme4
  cooks.distance.influence.merMod lme4
  dfbeta.influence.merMod         lme4
  dfbetas.influence.merMod        lme4


In [50]:
get_processed_data <- function(data_path){
    data <- read.table("measures.csv", header = TRUE, sep = ",")
    data$subj_id <- as.factor(data$subj_id)
    
    data$tta_condition <- scale(data$tta_condition)
    data$d_condition <- scale(data$d_condition)
    data$v_condition <- scale(data$v_condition)
    data$RT <- scale(data$RT)
    
    data$decision <- relevel(data$decision, ref=2)
  
    return(data)
}

data <- get_processed_data(data_path)

In [51]:
contrasts(data$decision) 

,Turn
Wait,0
Turn,1


In [38]:
data

subj_id,session,route,intersection_no,idx_bot_spawn,idx_response,idx_min_distance,min_distance,RT,tta_condition,d_condition,v_condition,is_turn_decision,decision
<fct>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,"<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>",<fct>,<fct>
129,1,1,5,114,161,471,3.226016,-0.8071028,-1.219098615,-1.26140999,-0.36052672,False,Wait
129,1,1,6,41,56,389,16.099580,-1.0229918,0.004526356,-1.26140999,-1.05247639,True,Turn
129,1,1,8,45,87,506,3.234011,-0.8398710,0.004526356,-1.26140999,-1.05247639,False,Wait
129,1,1,10,77,124,430,8.046337,-0.8073002,-1.219098615,1.19183582,1.94597215,True,Turn
129,1,1,11,84,92,380,48.915927,-1.0708940,1.228151327,-0.03478709,-0.74494320,True,Turn
129,1,1,13,49,111,414,33.930989,-0.7054421,1.228151327,-0.03478709,-0.74494320,True,Turn
129,1,1,14,0,68,393,3.625540,-0.6266798,-1.219098615,1.19183582,1.94597215,False,Wait
129,1,1,15,57,102,403,7.893319,-0.8208549,-1.219098615,-0.03478709,0.79272271,True,Turn
129,1,1,19,59,125,459,12.732931,-0.6784642,0.004526356,-0.03478709,-0.12987684,True,Turn


# Decision as a function of TTA and d

In [55]:
rnd_effects_analysis_decision <- function(data){
    rnd1 = glmer(decision ~ tta_condition + d_condition + v_condition + (1|subj_id), data, family = binomial)
    rnd2 = glmer(decision ~ tta_condition + d_condition + v_condition + (d_condition|subj_id), data, family = binomial)
    rnd3 = glmer(decision ~ tta_condition + d_condition + v_condition + (tta_condition|subj_id), data, family = binomial)
    rnd4 = glmer(decision ~ tta_condition + d_condition + v_condition + (d_condition + tta_condition|subj_id), data, family = binomial)
    
    rnd.anova = anova(rnd1, rnd2, rnd3, rnd4)
    print(rnd.anova)
    
    print("Best model according to AIC")
    print(row.names(rnd.anova[rnd.anova$AIC==min(rnd.anova$AIC), ]))
    print("Best model according to BIC")
    print(row.names(rnd.anova[rnd.anova$BIC==min(rnd.anova$BIC), ]))
}

In [56]:
rnd_effects_analysis_decision(data)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.00105508 (tol = 0.001, component 1)"

Data: data
Models:
rnd1: decision ~ tta_condition + d_condition + v_condition + (1 | subj_id)
rnd2: decision ~ tta_condition + d_condition + v_condition + (d_condition | 
rnd2:     subj_id)
rnd3: decision ~ tta_condition + d_condition + v_condition + (tta_condition | 
rnd3:     subj_id)
rnd4: decision ~ tta_condition + d_condition + v_condition + (d_condition + 
rnd4:     tta_condition | subj_id)
     Df    AIC    BIC  logLik deviance  Chisq Chi Df Pr(>Chisq)  
rnd1  5 791.27 814.76 -390.64   781.27                           
rnd2  7 787.13 820.01 -386.56   773.13 8.1472      2    0.01702 *
rnd3  7 790.74 823.63 -388.37   776.74 0.0000      0    1.00000  
rnd4 10 789.22 836.21 -384.61   769.22 7.5193      3    0.05706 .
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
[1] "Best model according to AIC"
[1] "rnd2"
[1] "Best model according to BIC"
[1] "rnd1"


In [49]:
fitted_model = glmer(decision ~ tta_condition + d_condition + (d_condition | subj_id), data, family = binomial)
print(summary(fitted_model))

output = summary(fitted_model)$coefficients

file_name = file.path("figures", "decision.tex")
print(xtable(output, display = c("g","g","g","g","g"), # digits = c(2,3,2,2,2,-2), 
             label = "tab:decision", 
             caption = "Results of statistical analysis of the effect of distance and TTA conditions on decision"), 
      caption.placement = "top", table.placement="!h", math.style.exponents = TRUE, floating.environment = "table*", 
      type = "latex", file = file_name, booktabs = TRUE)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: decision ~ tta_condition + d_condition + (d_condition | subj_id)
   Data: data

     AIC      BIC   logLik deviance df.resid 
   785.2    813.4   -386.6    773.2      805 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.6310 -0.5055 -0.1712  0.5844  4.4666 

Random effects:
 Groups  Name        Variance Std.Dev. Corr 
 subj_id (Intercept) 0.3501   0.5917        
         d_condition 0.1121   0.3348   -0.87
Number of obs: 811, groups:  subj_id, 7

Fixed effects:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -0.49603    0.24350  -2.037   0.0416 *  
tta_condition  0.96310    0.09831   9.796   <2e-16 ***
d_condition    1.38070    0.16751   8.243   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
            (Intr) tt_cnd
tta_conditn -0.049       
d_condition -0.6

In [53]:
coef(fitted_model)

$subj_id
    (Intercept) tta_condition d_condition
129 -0.15147236     0.9630951   1.1147480
389 -0.84104330     0.9630951   1.6034618
525  0.34748810     0.9630951   0.8799147
616 -1.35807972     0.9630951   1.7250922
618 -0.37739440     0.9630951   1.2875546
642 -0.92080051     0.9630951   1.6464302
755 -0.08973689     0.9630951   1.3413878

attr(,"class")
[1] "coef.mer"

# RT as a function of TTA and distance

In [51]:
rnd_effects_analysis_RT <- function(data){
    rnd1 = lmer(RT ~ tta_condition + d_condition + (1|subj_id), data)
    rnd2 = lmer(RT ~ tta_condition + d_condition + (d_condition|subj_id), data)
    rnd3 = lmer(RT ~ tta_condition + d_condition + (tta_condition|subj_id), data)
    rnd4 = lmer(RT ~ tta_condition + d_condition + (d_condition + tta_condition|subj_id), data)
    
    rnd.anova = anova(rnd1, rnd2, rnd3, rnd4)
    print(rnd.anova)
    
    print("Best model according to AIC")
    print(row.names(rnd.anova[rnd.anova$AIC==min(rnd.anova$AIC), ]))
    print("Best model according to BIC")
    print(row.names(rnd.anova[rnd.anova$BIC==min(rnd.anova$BIC), ]))
}

In [52]:
rnd_effects_analysis_RT(data[data$decision=="Turn",])

boundary (singular) fit: see ?isSingular
boundary (singular) fit: see ?isSingular
boundary (singular) fit: see ?isSingular
refitting model(s) with ML (instead of REML)


Data: data
Models:
rnd1: RT ~ tta_condition + d_condition + (1 | subj_id)
rnd2: RT ~ tta_condition + d_condition + (d_condition | subj_id)
rnd3: RT ~ tta_condition + d_condition + (tta_condition | subj_id)
rnd4: RT ~ tta_condition + d_condition + (d_condition + tta_condition | 
rnd4:     subj_id)
     Df     AIC     BIC logLik deviance  Chisq Chi Df Pr(>Chisq)    
rnd1  5 -308.69 -289.41 159.35  -318.70                             
rnd2  7 -305.40 -278.40 159.70  -319.40 0.7058      2     0.7027    
rnd3  7 -308.83 -281.83 161.42  -322.83 3.4335      0     <2e-16 ***
rnd4 10 -302.97 -264.39 161.49  -322.97 0.1358      3     0.9872    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
[1] "Best model according to AIC"
[1] "rnd3"
[1] "Best model according to BIC"
[1] "rnd1"


Singular fit for rnd3, so using random intercept only

In [54]:
fitted_model = lmer(RT ~ tta_condition + d_condition + (1 | subj_id), data[data$decision=="Turn",])
print(summary(fitted_model))

output = summary(fitted_model)$coefficients

file_name = file.path("figures", "RT.tex")
print(xtable(output, display = c("g","g","g","g","g","g"), # digits = c(2,3,2,2,2,-2), 
             label = "tab:RT", 
             caption = "Results of statistical analysis of the effect of distance and TTA conditions on response time"), 
      caption.placement = "top", table.placement="!h", math.style.exponents = TRUE, floating.environment = "table*", 
      type = "latex", file = file_name, booktabs = TRUE)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: RT ~ tta_condition + d_condition + (1 | subj_id)
   Data: data[data$decision == "Turn", ]

REML criterion at convergence: -298.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.3888 -0.6064 -0.0964  0.4145  5.1569 

Random effects:
 Groups   Name        Variance Std.Dev.
 subj_id  (Intercept) 0.006613 0.08132 
 Residual             0.022500 0.15000 
Number of obs: 350, groups:  subj_id, 7

Fixed effects:
                Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)    -0.721750   0.032610   6.770557 -22.133 1.44e-07 ***
tta_condition   0.028214   0.008940 342.485172   3.156  0.00174 ** 
d_condition    -0.005228   0.009983 343.831637  -0.524  0.60085    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
            (Intr) tt_cnd
tta_conditn -0.135       
d_condition -0.193  0.165


In [55]:
coef(fitted_model)

$subj_id
    (Intercept) tta_condition  d_condition
129  -0.5974069    0.02821421 -0.005227951
389  -0.7201394    0.02821421 -0.005227951
525  -0.8578226    0.02821421 -0.005227951
616  -0.7063182    0.02821421 -0.005227951
618  -0.7156756    0.02821421 -0.005227951
642  -0.6909007    0.02821421 -0.005227951
755  -0.7639880    0.02821421 -0.005227951

attr(,"class")
[1] "coef.mer"